<a href="https://colab.research.google.com/github/sljm12/machine_learning_notebooks/blob/master/LLM/Transcript_Q%26A_with_Gradio_and_VTTSplitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Youtube Transcript Q&A

Using the national day rally speech from https://www.youtube.com/watch?v=LCcWWbx6pXU

Prototype of a Youtube video Q&A with the ability to show the video segments in which the data is taken from from.

Youtube Subtitle Download script from https://github.com/ithelpTheDoctor/Code-Examples/blob/main/Youtube%20Subtitle%20Downloader/subtitles_downloader.py

In [1]:
!pip install yt-dlp auto-gptq huggingface_hub sentencepiece chromadb langchain pypdf sentence_transformers tqdm gradio>/dev/null

In [2]:
!pip install git+https://github.com/sljm12/langchain_vttsplitter >/dev/null

  Running command git clone --filter=blob:none --quiet https://github.com/sljm12/langchain_vttsplitter /tmp/pip-req-build-vbszmuaf


# Loading the Model

## Cloning the Vicuna model from TheBloke


In [3]:
#model_dir = "/content/vicuna-7B-v1.5-GPTQ"
model_dir = "/content/vicuna-13B-v1.5-16K-GPTQ"

In [4]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="TheBloke/vicuna-13B-v1.5-16K-GPTQ", local_dir=model_dir)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

'/content/vicuna-13B-v1.5-16K-GPTQ'

## Loading the model and tokenizer

In [5]:
from transformers import AutoTokenizer, TextGenerationPipeline
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

quantize_config = BaseQuantizeConfig(
        bits=4,
        group_size=128,
        desc_act=False
    )

tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model= AutoGPTQForCausalLM.from_quantized(model_dir, device="cuda:0",use_safetensors=True,use_triton=False)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Vicuna 7B ~ 5GB
Vicuna 13B 14GB

In [6]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!nvidia-smi

Tue Oct  3 09:04:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    26W /  70W |   8931MiB / 15360MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Setting up the text generation pipeline

In [7]:
RETURN_FULL_TEXT = False #If you set this to true it will return the prompt text also
pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer,max_new_tokens=512, return_full_text=RETURN_FULL_TEXT)

The model 'LlamaGPTQForCausalLM' is not supported for . Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMH

In [8]:
VICUNA_TEMPLATE ="""
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

USER:

{prompt}

ASSISTANT:

"""

In [9]:
from langchain.chains.retrieval_qa.prompt import PROMPT, prompt_template

QA_PROMPT = VICUNA_TEMPLATE.format(prompt=prompt_template)

In [10]:
prompt_template="""
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
"""
QA_PROMPT = VICUNA_TEMPLATE.format(prompt=prompt_template)

In [11]:
print(QA_PROMPT)


A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

USER:


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}


ASSISTANT:




# Setting up ChromaDB imports and sentence embeddings

In [12]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [14]:
# Youtube player iframe
iframe_yt = """
<tr>
<td>
<iframe id="ytplayer" type="text/html" width="640" height="360"
  src="{yt_url}&cc_load_policy=1"
  frameborder="0" ></iframe>
</td>
<td>
{transcript}
</td>
</tr>
"""

In [15]:
table_yt = """
<table>
{rows}
</table>
"""

In [16]:
# The chromaDB variable we put it here so that even when the gradio app stops we can continue to query the DB
db = None

# The Gradio app

In [17]:
import gradio as gr, os, glob, subprocess,time, datetime,urllib.parse,requests
from langchain.schema.document import Document
from langchain_vttsplitter.vttsplitter import YoutubeSubtitleLoader, VTTSplitter


def query(question):
  global db
  ans=db.similarity_search_with_score(question)
  context="\n".join([i[0].page_content+'\n' for i in ans])
  return pipeline(QA_PROMPT.format(question=question, context=context)), ans

def load_db(url):
  global db
  if db is not None:
    db.delete_collection()

  msg = ""
  try:
    vtt_doc = YoutubeSubtitleLoader(youtube_url=url,language='en').load()
    docs = VTTSplitter().split_text_docs(vtt_doc[0])
    db = Chroma.from_documents(docs, embeddings,collection_metadata={"url":url})
    done_fmt = "Done: {url} {date} Collection Count: {collection_count}"
    msg = done_fmt.format(url=url,
                          date=datetime.datetime.now().isoformat()
                          ,collection_count=str(db._collection.count()))

  except ValueError:
    msg = "Error loading "+url

  return msg

def download(u):
  global db
  return load_db(u)

def extract_embed_url(link,start_time,end_time):
  qs=urllib.parse.urlparse(link).query
  params=urllib.parse.parse_qs(qs)
  return "https://youtube.com/embed/{id}?start={start}&end={end}".format(id=params["v"][0],
                                                                         start=int(start_time),
                                                                         end=int(end_time))

def return_html_iframe(links, transcripts):
  rows = "\n".join([iframe_yt.format(yt_url=l,transcript=transcripts[i]) for i,l in enumerate(links)])
  return table_yt.format(rows=rows)

def query_click(query):
  global db
  ans=db.similarity_search_with_score(query)
  context="\n".join([i[0].page_content+'\n' for i in ans])
  transcripts = [i[0].page_content for i in ans]
  video_ids = [extract_embed_url(i[0].metadata["url"],i[0].metadata["start_time"],i[0].metadata["end_time"]) for i in ans]
  return pipeline(QA_PROMPT.format(question=query, context=context))[0]["generated_text"],return_html_iframe(video_ids, transcripts)

with gr.Blocks() as demo:
  with gr.Row():
    youtube=gr.Textbox(label="Youtube URL", value ="https://www.youtube.com/watch?v=LCcWWbx6pXU", scale=3)
    load_youtube=gr.Button(value="Load Youtube", scale=1)


  youtube_output=gr.Textbox(label="Loading")
  load_youtube.click(download,inputs=[youtube],outputs=[youtube_output])

  with gr.Row():
    query_text=gr.Textbox(label="Query",value="What are the initiatives in the rally?",scale=3)
    query_btn=gr.Button(value="Submit Query", scale=1)


  output=gr.Textbox(lines=7,label="Output")

  html_1=gr.HTML()
  query_btn.click(query_click,inputs=[query_text],outputs=[output,html_1])

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8acec15615d37c0312.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[youtube] Extracting URL: https://www.youtube.com/watch?v=l5Kpkx4ZfRA
[youtube] l5Kpkx4ZfRA: Downloading webpage
[youtube] l5Kpkx4ZfRA: Downloading ios player API JSON
[youtube] l5Kpkx4ZfRA: Downloading android player API JSON
[youtube] l5Kpkx4ZfRA: Downloading m3u8 information
[info] l5Kpkx4ZfRA: Downloading subtitles: en
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8acec15615d37c0312.gradio.live
